In [15]:

import pandas as pd
import numpy as np
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Hàm load dữ liệu theo batch
def load_data_by_batch(csv_path, batch_id):
    df = pd.read_csv(csv_path)
    df["Bộ"] = df["Bộ"].astype(int)  # Chuyển về kiểu số nguyên nếu cần

    batch_df = df[df["Bộ"] == batch_id]

    n_fft = batch_df["n_fft"].values[0]
    hop_length = batch_df["hop_length"].values[0]

    # Load train data
    train_row = batch_df[batch_df["Tập"] == "train"]
    test_row = batch_df[batch_df["Tập"] == "test"]

    train_x_path = train_row["X_file_path"].values[0]
    train_y_path = train_row["y_file_path"].values[0]
    test_x_path = test_row["X_file_path"].values[0]
    test_y_path = test_row["y_file_path"].values[0]

    # Đọc dữ liệu
    X_train = np.load(train_x_path)
    y_train = np.load(train_y_path)
    X_test = np.load(test_x_path)
    y_test = np.load(test_y_path)

    return X_train, y_train, X_test, y_test, n_fft, hop_length

In [16]:
# Hàm chạy SVM với nhiều mức MFCC (13, 20, 40, 80)
def run_rf_experiment(X_train, y_train, X_test, y_test, n_fft, hop_length, mfcc_steps):
    scaler = StandardScaler()

    # Scale toàn bộ 80 MFCC trước
    num_samples_train = X_train.shape[0]
    num_samples_test = X_test.shape[0]

    X_train_scaled = scaler.fit_transform(X_train.reshape(num_samples_train, -1))
    X_test_scaled = scaler.transform(X_test.reshape(num_samples_test, -1))
    print(X_train_scaled.shape)
    print(X_test_scaled.shape)

    rf = RandomForestClassifier(random_state=42)
    rf_params = {
        'n_estimators': [100, 200, 300, 400],
        'max_depth': [10, 20, 30, 40, 50],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }


    results = []
    for i in mfcc_steps:
        # Lấy i hệ số MFCC đầu tiên
        X_train_subset = X_train_scaled[:, :i]
        X_test_subset = X_test_scaled[:, :i]

        # Đo thời gian chạy
        start_time = time.time()

        # random search
        random_search = RandomizedSearchCV(rf, rf_params, n_iter=10, cv=3, random_state=42, n_jobs=-1, verbose=1)
        random_search.fit(X_train_subset, y_train)


        best_params = random_search.best_params_
        y_pred = random_search.predict(X_test_subset)
        test_accuracy = accuracy_score(y_test, y_pred)
        macro_f1 = f1_score(y_test, y_pred, average='macro')
        confusion = confusion_matrix(y_test,y_pred)
        print(f"MFCC: {i}")
        print(confusion)

        elapsed_time = time.time() - start_time
       

        # Lưu kết quả
        results.append({
            'numofMFCC': i,
            'n_fft': n_fft,
            'Hop_length': hop_length,
            'Best_hyperparameters': best_params,
            'test_accuracy': test_accuracy,
            'macro_f1': macro_f1,
            'time': elapsed_time
        })

    return pd.DataFrame(results)

In [17]:
# Chạy thử nghiệm
mfcc_steps = [13, 20, 40, 80]
csv_path = r"C:\Users\manhm\Desktop\BeeSoundClassifier\data\extracted_features\mfcc\mfcc_extraction_log.csv"

for i in range(0, 4):
    batch_id = i
    data = load_data_by_batch(csv_path, batch_id)
    
    if data is None:
        continue  # Nếu batch không có dữ liệu thì bỏ qua
    print(f"\n📂 Batch {batch_id}" )
    print("Training")
    X_train, y_train, X_test, y_test, n_fft, hop_length = data
    df = run_rf_experiment(X_train, y_train, X_test, y_test, n_fft, hop_length, mfcc_steps)

    print(f"\n📌 Kết quả Batch {batch_id}:")
    print(df)

    df.to_csv(f"rf_results_batch_{batch_id}.csv", index=False)



📂 Batch 0
Training
(11032, 80)
(2760, 80)
Fitting 3 folds for each of 10 candidates, totalling 30 fits
MFCC: 13
[[1028   58    9]
 [ 164  319  209]
 [   2   34  937]]
Fitting 3 folds for each of 10 candidates, totalling 30 fits
MFCC: 20
[[1038   53    4]
 [ 155  327  210]
 [   1   28  944]]
Fitting 3 folds for each of 10 candidates, totalling 30 fits
MFCC: 40
[[1047   46    2]
 [ 155  329  208]
 [   0   26  947]]
Fitting 3 folds for each of 10 candidates, totalling 30 fits
MFCC: 80
[[1047   47    1]
 [ 163  312  217]
 [   0   23  950]]

📌 Kết quả Batch 0:
   numofMFCC  n_fft  Hop_length  ... test_accuracy  macro_f1       time
0         13   1024         256  ...      0.827536  0.785757  32.000356
1         20   1024         256  ...      0.836594  0.795820  53.446864
2         40   1024         256  ...      0.841667  0.800946  75.814590
3         80   1024         256  ...      0.836594  0.792300  96.981214

[4 rows x 7 columns]

📂 Batch 1
Training
(11032, 80)
(2760, 80)
Fitting 3 fo